# Inversion Package Development
## Tyler Witt & Kieran Yanaway

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
def calc_latitude_radius(lat):
    """
    Calculate the radius of Earth at a specific latitude
    _____________________________________________________________________
    lat : float : radians : latitude at which to calculate Earth's radius
    _____________________________________________________________________
    """
    EQ_RADIUS = 6378.137 # Earth's equatorial radius, km
    PO_RADIUS = 6356.752 # Earth's polar radius, km

    return np.sqrt( ((EQ_RADIUS ** 2 * np.cos(lat)) ** 2 + (PO_RADIUS ** 2 * np.sin(lat)) ** 2) /
                      ((EQ_RADIUS * np.cos(lat)) ** 2 + (PO_RADIUS * np.sin(lat)) ** 2) ) # calculate radius using [1] equation B.3.5.8

def haversine(theta):
    """
    Calculate the haversine of an angle
    _____________________________________________________________
    theta : float : radians : angle to calculate the haversine of
    _____________________________________________________________
    """
    return np.sin(theta / 2) ** 2

def haversine_formula(lat1, long1, lat2, long2):
    """
    Calculate the haversine of the central angle between two lat-long coordinates
    ___________________________________________________________________
    lat1  : float : radians latitude  : latitudinal position of point 1
    long1 : float : radians longitude : longitudinal position of point 1
    lat2  : float : radians latitude  : latitudinal position of point 2
    long2 : float : radians longitude : longitudinal position of point 2
    ____________________________________________________________________
    """
    dphi = lat2 - lat1
    dlam = long2 - long1
    return haversine(dphi) + np.cos(lat1) * np.cos(lat2) * haversine(dlam)

def great_circle_distance(lat1, long1, lat2, long2, r):
    """
    Calculate the great circle distance between two lat-long coordinates
    ___________________________________________________________________
    lat1  : float : radians latitude  : latitudinal position of point 1
    long1 : float : radians longitude : longitudinal position of point 1
    lat2  : float : radians latitude  : latitudinal position of point 2
    long2 : float : radians longitude : longitudinal position of point 2
    r     : float : kilometers        : radius of Earth in desired area
    ___________________________________________________________________
    """
    return 2 * r * np.arcsin(np.sqrt(haversine_formula(lat1, long1, lat2, long2)))

In [3]:
class Trajectory:
    """
    Trajectory Class
    ----------------
    A Trajectory object wraps the parameters returned by a TrajectoryEstimator object after running a trajectory inversion.
    It will also contain some functionality for visualization of the parameters, as well as statistics from the inversion.
    """

    def __init__(self, x0, y0, t0, v, theta, gamma):
        """
        Initialize a Trajectory object with the inversion parameters.
        ________________________________________________________________________________________________
        x0    : float : degrees latitude  : x position of the trajectory intersection with the x-y plane
        y0    : float : degrees longitude : y position of the trajectory intersection with the x-y plane
        t0    : float : seconds           : time of the trajectory intersection with the x-y plane
        v     : float : kilometers/second : velocity of the meteoroid
        theta : float : radians           : elevation angle of the trajectory
        gamma : float : radians           : azimuthal angle of the trajectory (measured anticlockwise from South)
        _________________________________________________________________________________________________________
        """
        self.x0 = x0
        self.y0 = y0
        self.t0 = t0
        self.v = v
        self.theta = theta
        self.gamma = gamma
        self.lat = None
        self.long = None
    
    def print_parameters(self):
        print("x0:", self.x0, "kilometers")
        print("y0:", self.y0, "kilometers")
        print("t0:", self.t0, "seconds")
        print("velocity:", self.v, "km/s")
        print("theta:", np.rad2deg(self.theta), "degrees")
        print("gamma:", np.rad2deg(self.gamma), "degrees")

In [4]:
class TrajectoryGridsearchParameters:
    """
    TrajectoryGridsearchParameters class
    ------------------------------------
    A TrajectoryGridsearchParameters class easily formats the inversion gridsearch parameters for
    interface with a TrajectoryEstimator object.
    """

    def __init__(self,x0p=(-300,300,10),y0p=(-300,300,10),t0p=(0,100,10),vep=(11.91015,81.6696,10),thp=(0,90,10),gap=(0,360,10)):
        """
        Initialize a TrajectoryGridsearchParameters object with the desired parameter ranges for a gridsearch
        __________________________________________________________________________________________
        x0p : (start, stop, n) : (kilometers, kilometers, integer) : range of x0s to search across
        y0p : (start, stop, n) : (kilometers, kilometers, integer) : range of y0s to search across
        t0p : (start, stop, n) : (kilometers, kilometers, integer) : range of t0s to search across
        vep : (start, stop, n) : (km/s, km/s, integer)             : range of velocities to search across
        thp : (start, stop, n) : (degrees, degrees, integer)       : range of thetas to search across
        gap : (start, stop, n) : (degrees, degrees, integer)       : range of gammas to search across
        _____________________________________________________________________________________________
        """
        self.x0s = None
        self.y0s = None
        self.t0s = None
        self.velocities = None
        self.thetas = None
        self.gammas = None
        
        self.set_x0p(x0p)
        self.set_y0p(y0p)
        self.set_t0p(t0p)
        self.set_vep(vep)
        self.set_thp(thp)
        self.set_gap(gap)
        
    def set_x0p(self, x0p):
        x0min = x0p[0]
        x0max = x0p[1]
        x0disc = x0p[2]
        self.x0s = np.linspace(x0min,x0max,x0disc)
        
    def set_y0p(self, y0p):
        y0min = y0p[0]
        y0max = y0p[1]
        y0disc = y0p[2]
        self.y0s = np.linspace(y0min,y0max,y0disc)

    def set_t0p(self, t0p):
        t0min = t0p[0]
        t0max = t0p[1]
        t0disc = t0p[2]
        self.t0s = np.linspace(t0min,t0max,t0disc)

    def set_vep(self, vep):
        vemin = vep[0]
        vemax = vep[1]
        vedisc = vep[2]
        self.velocities = np.linspace(vemin,vemax,vedisc)

    def set_thp(self, thp):
        thmin = thp[0]
        thmax = thp[1]
        thdisc = thp[2]
        self.thetas = np.deg2rad(np.linspace(thmin,thmax,thdisc))
        
    def set_gap(self, gap):
        gamin = gap[0]
        gamax = gap[1]
        gadisc = gap[2]
        self.gammas = np.deg2rad(np.linspace(gamin,gamax,gadisc))

In [5]:
class Station:
    """
    Station Class
    -------------
    A Station object wraps the relevant data for a seismic station for ease of interfacing with a TrajectoryEstimator
    object.
    """

    def __init__(self, lat, long, elev, ta):
        """
        Initialize a Station object with the relevant positional and arrival time data.
        ______________________________________________________________________________
        lat  : float : degrees latitude  : latitudinal position of the seismic station
        long : float : degrees longitude : longitudinal position of the seismic station
        elev : float : kilometers        : elevation of the seismic station
        ta   : float : seconds           : arrival time of ballistic wave at seismic station
        ____________________________________________________________________________________
        """
        self.lat = lat
        self.long = long
        self.elev = elev
        self.ta = ta
        self.x = None
        self.y = None

    def calc_x_y(self, olat, olong):
        """
        Calculate the x (southing) and y (easting) position of the station relative to origin latitude and longitude.
        _____________________________________________________________________________________________
        olat  : float : degrees latitude  : latitudinal position of the origin of the inversion space
        olong : float : degrees longitude : longitudinal position of the origin of the inversion space
        ______________________________________________________________________________________________
        """

        midlat = np.deg2rad((self.lat + olat) / 2) # midpoint latitude for radius calculation
        
        radius = calc_latitude_radius(midlat)

        self.x = (np.deg2rad(olat - self.lat) * radius) # x position is arc length between latitudes (+x = southing)
        
        ysign = 1
        if self.long != olong: # if at different longitudes...
            ysign = ((self.long - olong) / np.abs(self.long - olong)) # set sign of y distance (+y = easting)
        if (self.long < -90 and olong > 90) or (self.long > 90 and olong < -90): # if longitudes are on opposite sides of 180th meridian...
            ysign *= -1 # negate ysign
        self.y = ysign * great_circle_distance(midlat, np.deg2rad(olong),
                                               midlat, np.deg2rad(self.long), radius) # y position is arclength between longitudes at midlat



In [6]:
class TrajectoryEstimator:
    """
    TrajectoryEstimator Class
    -------------------------
    A TrajectoryEstimator object handles inversion of the seismic data from a meteoroid's ballistic wave to estimate the
    trajectory of the meteoroid through the atmosphere.
    """

    def __init__(self, olat, olong, ot, stations=None):
        """
        Initialize a TrajectoryEstimator object with the origin latitude and longitude, as well as an array of Stations
        _____________________________________________________________________________________________
        olat     : float     : degrees latitude  : latitudinal position of the origin of the inversion space
        olong    : float     : degrees longitude : longitudinal position of the origin of the inversion space
        ot       : TIME      : seconds           : origin time of inversion space
        stations : arraylike : init_val = None   : array containing the seismic stations which will be used for the inversion
        _____________________________________________________________________________________________________________________
        """
        self.olat = olat
        self.olong = olong
        self.otime = ot
        self.stations = stations
        if stations is not None:
            for station in stations:
                station.calc_x_y(olat, olong)

    def set_stations(self, lats, longs, elevs, ts):
        """
        Set the stations array of the TrajectoryEstimator using lists containing station data
        __________________________________________________________________________________________
        lats  : arraylike : degrees latitude  : array containing latitudinal positions of stations
        longs : arraylike : degrees longitude : array containing longitudinal positions of stations
        elevs : arraylike : kilometers        : array containing elevational positions of stations
        ts    : arraylike : seconds           : array containing ballistic wave arrival times of stations
        _________________________________________________________________________________________________ 
        """
        self.stations = np.array([None] * len(lats), dtype=Station)
        for i in range(len(lats)):
            self.stations[i] = Station(lats[i], longs[i], elevs[i], ts[i])
            self.stations[i].calc_x_y(self.olat, self.olong)
    
    def gridsearch(self,gsp):

        c = 0.32 # velocity of sound in atmosphere (km/s)

        minsq = float('inf') # set initial value for least squares
        optimalTrajectory = Trajectory

        p_track = 0 # percentage tracker
        for theta in gsp.thetas:
            print(str(round(100 * p_track, 2)) + "% complete")
            p_track += 1/gsp.thetas.size
            for gamma in gsp.gammas:
                A = np.array([[np.cos(gamma) * np.sin(theta), np.sin(gamma) * np.sin(theta), -1*np.cos(theta)],
                              [-1*np.sin(gamma), np.cos(gamma), 0],
                              [np.cos(gamma) * np.cos(theta), np.sin(gamma) * np.cos(theta), np.sin(theta)]]) # Rotation matrix for current parameters
                for velocity in gsp.velocities:
                    beta = np.arcsin(c / velocity)
                    for x0 in gsp.x0s:
                        for y0 in gsp.y0s:
                            for t0 in gsp.t0s:
                                sq_sum = 0 # sum of squares of residuals
                                for station in self.stations:
                                    b = np.array([[station.x - x0],
                                                  [station.y - y0],
                                                  [station.elev]])
                                    X, Y, Z = (A @ b).flatten() # rotated coordinate system with origin at meteoroid-surface intersection and Z along trajectory
    
                                    
                                    ti = t0 + (((np.sqrt(X*X + Y*Y) / np.tan(beta)) - Z) / velocity) # wave arrival time for current parameters
                                    sq_sum += (ti - station.ta) ** 2 # add square of residual
                                    
                                if sq_sum < minsq:
                                    #print(minsq)
                                    minsq = sq_sum
                                    optimalTrajectory = Trajectory(x0,y0,t0,velocity,theta,gamma) # update optimal trajectory if necessary

        return optimalTrajectory
    
    def invert_data(self, params=None, method="gridsearch"):
        optimalTrajectory = Trajectory
        
        if method == "gridsearch":
            if params is None:
                params = TrajectoryGridsearchParameters()
            optimalTrajectory = self.gridsearch(params)

        return optimalTrajectory
        

In [7]:
lats = np.array([39.387,39.473,39.355,39.632,39.118,39.466,39.374,39.237,39.618,39.403,38.952,39.486,38.984,39.616,39.848,39.086,39.256,39.130,39.396,39.061,38.857])
longs = np.array([141.57,141.29,141.20,141.30,141.26,141.00,140.94,140.91,141.08,140.77,141.22,140.79,141.04,140.90,141.24,140.72,140.63,140.66,140.63,140.59,140.72])
elevs = np.array([375,210,65,311,165,245,300,610,300,280,70,445,123,270,650,400,170,280,200,235,440]) / 1000
times = np.array([180 + 42.74,240 + 7.83,240 + 13.78,240 + 22.06,240 + 28.93,240 + 37.34,240 + 38.35,240 + 41.29,240 + 43.85,240 + 57.55,240 + 59.12,300 + 1.14,300 + 1.5,300 + 3.98,300 + 6.76,300 + 7.38,300 + 8.64,300 + 9.76,300 + 12.09,300 + 20.42,300 + 39.11])

m = TrajectoryEstimator(39,141,0)
m.set_stations(lats,longs,elevs,times)

In [8]:
for stat in m.stations:
    print(stat.x, stat.y)

-43.02325014479127 49.11081748025858
-52.583842818881315 24.97087710920566
-39.46580827612202 17.235829502499733
-70.25969806111715 25.80252786598336
-13.118301349838337 22.444444628071466
-51.805656806196296 0.0
-41.578040172843075 -5.170048247694088
-26.347684989168663 -7.762655324841743
-68.70333992545241 6.88136818224487
-44.80196863134954 -19.81440756307657
5.336283520506901 19.01385066154349
-54.029044584767824 -18.080680863942025
1.7787595453723126 3.4562801789283317
-68.48100294680395 -8.601833479941389
-94.2719221116698 20.60999626913587
-9.560804654738044 -24.176437655655935
-28.45993117127496 -31.908796944354208
-14.452360930210597 -29.34791607403421
-44.02377949428975 -31.876932202370778
-6.781505827413359 -35.40748463582994
15.897721177044867 -24.21573219557786


In [ ]:

traj = m.invert_data(TrajectoryGridsearchParameters((-100,-80,21),(200,220,21),(40,60,21),(10,30,21),(15,25,21),(280,295,31)))#(-95,-91,5),(205,209,5),(46,50,5),(16,20,5),(17.5,19.5,5),(286,288,5)))#(-110,-50,61),(180,250,71),(0,80,81),(11,45,35),(10,40,61),(270,300,61)))

0% complete


In [ ]:
traj.print_parameters()

In [ ]:
s=0
intime = time.time()
for i in range(10):
    for j in range(10):
        for k in range(10):
            for l in range(10):
                for m in range(10):
                    for o in range(10):
                        s += 0.0009765625

otime = time.time()

print(otime - intime, s)

[1] M. S. Grewal, Global navigation satellite systems, inertial navigation, and integration. 2020.
‌